In [ ]:
import pandas as pd
import nltk
import hashlib
import json
import csv
import ast

In [ ]:
# thefile = open('plg/fingerprints/Books_5.txt', 'r')
fingerprints = open('plg/fingerprints/Books_5.txt', 'r').readlines()

In [ ]:
thefile = open('plg/fingerprints/len_fp.txt', 'w')
for item in fplen1:
    if (i % 100000 == 0):
        print (i)
    thefile.write("%s\n" % item)

In [ ]:
fplen = open('plg/fingerprints/len_fp.txt', 'r').read().splitlines()


In [ ]:
len(fingerprints)

In [ ]:
num_reviews_batch = 10000

In [ ]:
for batch in range(30): # total 71.3
    plg_dic = {}
    print ("batch: ", batch)
    for index in range(num_reviews_batch):
        i = batch * num_reviews_batch + index
        for fp in eval(fingerprints[i]):
            if fp in plg_dic:
                indices = plg_dic[fp]
                indices.append(i)
                plg_dic[fp] = indices
            else:
                plg_dic[fp] = [i] 
    
    with open('plg/fingerprints/plg_dic.csv', 'a') as fp:
        writer = csv.writer(fp)
        for key, value in plg_dic.items():
           writer.writerow([key, value])

In [2]:
with open('plg/fingerprints/plg_dic.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    plg_dic = dict(reader)

KeyboardInterrupt: 

In [10]:
len(plg_dic)

20140870

In [11]:
plg_indices = []
for index, items in enumerate(plg_dic):
    if (index % 1000000 == 0):
        print (index)
    temp = ast.literal_eval(plg_dic[items])
    sz = len(temp)
    if (sz > 1):
        for i in range(sz):
            for j in range(i + 1, sz):
                plg_indices.append((temp[i], temp[j]))
            

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000


In [8]:
len(plg_indices)

163557

In [69]:
with open('plg/fingerprints/plg_indices_pairs.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in plg_indices))

In [ ]:
with open('plg/fingerprints/plg_indices_pairs.txt', 'r') as fp:
    plg_indices = [tuple(map(int, i.split(' '))) for i in fp]

In [5]:
plg_indices[2]

(274998, 275003)

In [ ]:
# thefile = open('plg/fingerprints/plg_indices_pairs.txt', 'w')
# for item in plg_indices:
#     thefile.write("%s %s\n" %item)

In [ ]:
# open with with take care of closing itself
# closing open files!!

In [18]:
unique_indices = set()
for i, pair in enumerate(plg_indices):
    unique_indices.add(pair[0])
    unique_indices.add(pair[1])

In [19]:
len(unique_indices)

89692

In [20]:
uni_fp_dic = {}
fp = open('plg/fingerprints/Books_5.txt', 'r')
for i, line in enumerate(fp):
    if i % 100000 == 0:
        print (i)
    if i in unique_indices:
        uni_fp_dic[i] = line

0
100000
200000
300000
400000
500000
600000
700000


In [21]:
with open('plg/fingerprints/uni_fp_dic.csv', 'w') as fp:
        writer = csv.writer(fp)
        for key, value in uni_fp_dic.items():
           writer.writerow([key, value])

In [2]:
with open('plg/fingerprints/uni_fp_dic.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    fingerprints = dict(reader)

In [3]:
len(fingerprints)

89692

In [ ]:
fingerprints[str(0)]

In [9]:
scores = []
sentence_threshold = 2 #minimum number of windows
for index, items in enumerate(plg_indices):
    if (index % 10000 == 0):
        print (index)
    i = items[0]
    j = items[1]
    common = eval(fingerprints[str(i)]) & eval(fingerprints[str(j)])
    temp = min(len(eval(fingerprints[str(i)])), len(eval(fingerprints[str(j)])))
    score = len(common)/temp
    if score > 0.7 and temp >= sentence_threshold:
        scores.append((i, j, score))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000


In [35]:
scores = set(scores)

In [36]:
suspected_indices = set()
for item in scores:
    suspected_indices.add(item[0])
    suspected_indices.add(item[1])

In [37]:
len(suspected_indices)

530

In [63]:
df = pd.read_pickle('data/data_frames/Books_5.pkl')

In [58]:
df.head()

,index,asin,helpful,overall,reviewerID
0,4,000100039X,"[7, 9]",5,A2XQ5LZHTD4AFT
1,23,000100039X,"[2, 9]",2,A1KQ80Y692CDOI
2,25,000100039X,"[5, 6]",5,AF7CSSGV93RXN
3,34,000100039X,"[81, 92]",5,A1NPNGWBVD9AK3
4,43,000100039X,"[8, 10]",5,A3IS4WGMFR4X65


In [65]:
df.iloc[21345]

index                    115
asin              0060535504
helpful             [43, 44]
overall                    5
reviewerID    A2ASEBCEN9ZWTZ
Name: 21345, dtype: object

In [66]:
helpfulness = df['helpful']
num_reviews = helpfulness.size
min_helpfull_review = 10
temp = [i for i in range(num_reviews)  if helpfulness[i][1] > min_helpfull_review]
df_temp = df.iloc[temp].reset_index()
df = df_temp

In [67]:
df.iloc[12117]

level_0                21331
index                  99982
asin              0060534397
helpful             [13, 14]
overall                    5
reviewerID    A1EFBLQG6XB271
Name: 12117, dtype: object

In [61]:
dftemp = df[df['index'] < df['level_0']]

In [62]:
dftemp

,level_0,index,asin,helpful,overall,reviewerID
12118,21345,115,0060535504,"[43, 44]",5,A2ASEBCEN9ZWTZ
12119,21346,116,0060535504,"[13, 13]",5,AUDSM2CTLLW1Q
12120,21350,134,0060535687,"[8, 15]",2,AGD9USHGQU9O1
12121,21351,137,0060535687,"[17, 17]",5,A2FNA5903D9E6Y
12122,21352,144,0060535687,"[14, 15]",5,A3JJUCA6K4P31H
12123,21355,158,0060535687,"[2, 13]",3,A269IOAF4PF76N
12124,21358,166,0060535687,"[16, 16]",4,A1KM816LI9VZ7M
12125,21362,177,0060535954,"[34, 34]",5,A1YRV8ICHSL41F
12126,21363,179,0060535954,"[0, 11]",2,A13MDVKGZR8IQ6
12127,21364,182,0060535954,"[18, 21]",5,A1XX37HBK45UG8


In [45]:
[i for i in scores if i[2] < .8]

[(240718, 240877, 0.75),
 (99849, 99861, 0.7857142857142857),
 (66163, 66189, 0.7941176470588235),
 (248897, 249451, 0.75),
 (147253, 147271, 0.7777777777777778),
 (91321, 91333, 0.7523809523809524),
 (24828, 24829, 0.7627118644067796),
 (243048, 243164, 0.736),
 (285373, 285539, 0.75),
 (143498, 143719, 0.704225352112676),
 (95336, 95346, 0.7058823529411765),
 (221144, 229420, 0.7222222222222222),
 (25364, 25637, 0.75),
 (295440, 295744, 0.7272727272727273),
 (191623, 191737, 0.7407407407407407),
 (291713, 291979, 0.7741935483870968),
 (272935, 274443, 0.7333333333333333),
 (191637, 191737, 0.717948717948718),
 (219920, 219921, 0.7058823529411765),
 (167208, 167214, 0.7647058823529411),
 (11299, 11300, 0.7045454545454546),
 (160064, 160115, 0.7619047619047619),
 (143506, 144276, 0.7272727272727273),
 (94475, 97673, 0.7777777777777778),
 (191997, 192119, 0.7162162162162162),
 (211142, 211527, 0.7055214723926381),
 (191623, 191637, 0.7129629629629629),
 (277375, 277415, 0.76363636363636

In [48]:
scores

{(1499, 1502, 0.9508196721311475),
 (3454, 3559, 1.0),
 (4138, 4140, 0.8421052631578947),
 (5900, 9167, 0.9230769230769231),
 (7672, 8967, 0.8974358974358975),
 (9045, 9120, 0.9347826086956522),
 (10255, 16244, 0.8446601941747572),
 (10255, 16300, 0.8875),
 (10259, 16311, 0.9076923076923077),
 (10800, 14820, 0.9090909090909091),
 (11299, 11300, 0.7045454545454546),
 (16793, 16881, 0.9245283018867925),
 (19495, 19685, 0.95),
 (19550, 19551, 0.9242424242424242),
 (20665, 26680, 1.0),
 (22436, 22438, 0.847457627118644),
 (24664, 24665, 1.0),
 (24828, 24829, 0.7627118644067796),
 (25003, 26444, 0.9777777777777777),
 (25364, 25637, 0.75),
 (25797, 26688, 1.0),
 (25797, 26701, 1.0),
 (26256, 29212, 0.8108108108108109),
 (26688, 26701, 1.0),
 (28256, 28257, 1.0),
 (30366, 30376, 0.967741935483871),
 (30567, 30569, 0.9745222929936306),
 (31153, 31157, 1.0),
 (34669, 34685, 0.98989898989899),
 (36255, 36256, 0.9921875),
 (39116, 39193, 1.0),
 (48310, 48311, 1.0),
 (51307, 53828, 0.8333333333333

In [40]:
unique_reviews = set()
for item in scores:
    i = item[0]
    j = item[1]
    review1 = df.iloc[i]
    review2 = df.iloc[j]
    if review1['reviewerID'] != review2['reviewerID'] and review1['asin'] != review2['asin']:
        unique_reviews.add((i, j, item[2]))

In [41]:
unique_reviews

set()

In [56]:
df.iloc[70401]

level_0              123714
index                 51995
asin             0141188936
helpful             [8, 15]
overall                   5
reviewerID    AMVZ4CIAWRC4E
Name: 70401, dtype: object

In [55]:
df.iloc[70695]

level_0              124255
index                 52988
asin             0141188936
helpful            [16, 39]
overall                   1
reviewerID    AO7MJ0AQIN0J9
Name: 70695, dtype: object

In [ ]:
i = 7966
j = 7967
common = fingerprints[i] & fingerprints[j]
temp = min(len(fingerprints[i]), len(fingerprints[j]))
print (len(common))
score = len(common)/temp
print (score)
print (fingerprints[i])
print (fingerprints[j])